## Target Encoding

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV  #Perforing grid search
import xgboost as xgb


from sklearn import base
from category_encoders import TargetEncoder
import gc
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
df_train=pd.read_csv('train.csv')
testDataX = pd.read_csv('test.csv')
trainX = df_train.iloc[:,1:-1]
trainY = df_train.iloc[:,-1]
testDataX = testDataX.iloc[:,1:]

In [8]:
trainX

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0,0,0,T,N,Red,Trapezoid,Snake,India,Oboe,...,7508f4ef1,e027decef,1,Contributor,Freezing,k,K,dh,3,8
299996,0,0,0,F,Y,Green,Trapezoid,Lion,Russia,Piano,...,397dd0274,80f1411c8,2,Novice,Freezing,h,W,MO,3,2
299997,0,0,0,F,Y,Blue,Star,Axolotl,Russia,Oboe,...,5d7806f53,314dcc15b,3,Novice,Boiling Hot,o,A,Bn,7,9
299998,0,1,0,F,Y,Green,Square,Axolotl,Costa Rica,Piano,...,1f820c7ce,ab0ce192b,1,Master,Boiling Hot,h,W,uJ,3,8


## ENCODE AND SPLIT

In [6]:
encoder = TargetEncoder()
encoder.fit(trainX,trainY)
train_Encoded = encoder.transform(trainX)
testX_Encoded = encoder.transform(testDataX)

In [7]:
train_Encoded

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,0.302537,0.290107,0.327145,0.360978,0.307162,0.242813,0.237743,...,0.372694,0.368421,2,0.403885,0.257877,0.306993,0.208354,0.401186,2,2
1,0,1,0,0.302537,0.290107,0.327145,0.290054,0.359209,0.289954,0.304164,...,0.189189,0.076924,1,0.403885,0.326315,0.206599,0.186877,0.303880,7,8
2,0,0,0,0.309384,0.290107,0.241790,0.290054,0.293085,0.289954,0.353951,...,0.223022,0.172414,1,0.317175,0.403126,0.306993,0.351864,0.206843,7,2
3,0,1,0,0.309384,0.290107,0.351052,0.290054,0.307162,0.339793,0.329472,...,0.325123,0.227273,1,0.403885,0.360961,0.330148,0.208354,0.355985,2,1
4,0,0,0,0.309384,0.333773,0.351052,0.290054,0.293085,0.339793,0.329472,...,0.376812,0.200000,1,0.403885,0.225214,0.206599,0.351864,0.404345,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0,0,0,0.302537,0.333773,0.351052,0.290054,0.307162,0.361311,0.329472,...,0.255814,0.340909,1,0.278533,0.225214,0.365212,0.266023,0.312962,3,8
299996,0,0,0,0.309384,0.290107,0.327145,0.290054,0.293085,0.289954,0.304164,...,0.287805,0.194444,2,0.242055,0.225214,0.306993,0.409481,0.224880,3,2
299997,0,0,0,0.309384,0.290107,0.241790,0.310627,0.319017,0.289954,0.329472,...,0.335616,0.300000,3,0.242055,0.360961,0.433657,0.186877,0.178437,7,9
299998,0,1,0,0.309384,0.290107,0.327145,0.338932,0.319017,0.314688,0.304164,...,0.278226,0.360000,1,0.355078,0.360961,0.306993,0.409481,0.438520,3,8


In [91]:
X_train,X_test,y_train,y_test=train_test_split(train_Encoded,trainY,random_state=42,test_size=0.2)


## Logistic Regression

In [92]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pre=lr.predict(X_test)
y_pre1 = lr.predict(X_test)
testY = lr.predict_proba(testX_Encoded)
testY

array([[0.78593715, 0.21406285],
       [0.4346459 , 0.5653541 ],
       [0.96447481, 0.03552519],
       ...,
       [0.56179993, 0.43820007],
       [0.46906432, 0.53093568],
       [0.78562749, 0.21437251]])

In [59]:
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_test.values, y_pre))

AUC Score (Train): 0.708616
AUC Score (test): 0.708616


In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['target'] = testY
submission.to_csv('TargetEncoding_LR.csv', index=False)

## XGB

TargetEncoding_XGB - base
TargetEncoding_XGB1 - n_estimator optimized
TargetEncoding_XGB2 - n_estimator stochastic optimized
TargetEncoding_XGB3 - n_estimator w stochastic L1 and L2 optimized

In [93]:
%%time
# Define the model
xgb = XGBClassifier(n_estimators=500, learning_rate=0.1)
# Fit the model
xgb.fit(X_train,y_train)
testY = xgb.predict(X_test)
predictions_1 = xgb.predict_proba(testX_Encoded)
predictions_1

Wall time: 1min 44s


array([[0.7500727 , 0.2499273 ],
       [0.3973807 , 0.6026193 ],
       [0.98535365, 0.01464636],
       ...,
       [0.5796592 , 0.42034078],
       [0.40053606, 0.59946394],
       [0.8322132 , 0.16778678]], dtype=float32)

In [94]:
print ("Accuracy : %.4g" % metrics.accuracy_score(y_test.values, testY))

Accuracy : 0.7815


In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['target'] = predictions_1
submission.to_csv('TargetEncoding_XGB.csv', index=False)

### Stochastic and L1 and L2 Parameter tuning

In [ ]:
## Function to test optimum parameters

def modelfit_predtest(alg, dtrain, target,trainy, trainy_true, test_real, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=target.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], 
                          nfold=cv_folds,metrics='auc', early_stopping_rounds=early_stopping_rounds, 
                          verbose_eval=None)
        print ("Best N-estimator result for current learning rate: ", cvresult.shape[0])
        ## Set best n-estimator in our model
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors],target,eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])
    #Predict test set
    dtest_predprob = alg.predict_proba(trainy[predictors])
    #Predict real Data set
    dreal_predprob = alg.predict_proba(test_real[predictors])    
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(target.values, dtrain_predictions))
   # print ("AUC Score (Train): %f" % metrics.roc_auc_score(target.values, dtrain_predprob))
    #print ("AUC Score (Test): %f" % metrics.roc_auc_score(trainy_true.values, dtest_predprob))
    return dtrain_predictions, dtrain_predprob, dtest_predprob, dreal_predprob ,
                    
    #feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    #feat_imp.plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')

### Find out best n_estimators for learning rate = 0.1 and set it with cv func of xgb

In [ ]:
## Pre-set XGB Classifier parameters

predictors = list(X_train)
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
## Fixing learning rate 
train_acc, train_pred, test_pred, real_pred = modelfit_predtest(xgb1, X_train, y_train, X_test, y_test,testX_Encoded, predictors)

In [ ]:
print ("\nModel Report")
print ("Accuracy : %.4g" % metrics.accuracy_score(y_train.values, train_acc))
real_pred
#real_pred

In [86]:
submission = pd.read_csv('sample_submission.csv')
submission['target'] = real_pred
submission.to_csv('TargetEncoding_XGB1.csv', index=False)

In [ ]:
# XGBoost on Otto dataset, tune subsample
from pandas import read_csv
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

subsample = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]
param_grid = dict(subsample=subsample)
# grid search
model = XGBClassifier()
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model,param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
# XGBoost on Otto dataset, tune subsample
from pandas import read_csv
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

col_subsample = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]
param_grid = dict(subsample=col_subsample)
# grid search
model = XGBClassifier()
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model,param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(train, train_y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))


## L1 and L2 regularization

In [ ]:
from matplot import pyplot
import matplotlib
matplotlib.use('Agg')
# plot
pyplot.errorbar(subsample, means, yerr=stds)
pyplot.title("XGBoost subsample vs Log Loss")
pyplot.xlabel('subsample')
pyplot.ylabel('Log Loss')
pyplot.savefig('subsample.png')

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['target'] = predictions_1
submission.to_csv('TargetEncoding_XGB.csv', index=False)

## Random Forest

In [ ]:
%%time
forest_model = RandomForestRegressor(random_state=1,max_depth=20)
forest_model.fit(train, train_y)
rand_preds = forest_model.predict(test)
rand_preds
#print(mean_absolute_error(val_y, melb_preds))

In [ ]:
pd.DataFrame({'id': test_id, 'target': rand_preds}).to_csv(
    'targetEncoding_RF.csv', 
    index=False)

## Random Forest

In [106]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV  #Perforing grid search
import category_encoders as ce
import gc
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


In [99]:
#print(train.target.value_counts()[0]/300000, train.target.value_counts()[1]/300000, )
train=pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.sort_index(inplace=True)
train_y = train['target']
test_id = test['id']
train.drop(['target', 'id'], axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [100]:
cat_feat_to_encode = train.columns.tolist()
smoothing=0.50
oof = pd.DataFrame([])
for tr_idx, oof_idx in StratifiedKFold(n_splits=5, random_state=1, shuffle=True).split(train, train_y):
    ce_target_encoder = ce.TargetEncoder(cols = cat_feat_to_encode, smoothing=smoothing)
    ce_target_encoder.fit(train.iloc[tr_idx, :], train_y.iloc[tr_idx])
    oof = oof.append(ce_target_encoder.transform(train.iloc[oof_idx, :]), ignore_index=False)
ce_target_encoder = ce.TargetEncoder(cols = cat_feat_to_encode, smoothing=smoothing)
ce_target_encoder.fit(train, train_y)
train = oof.sort_index() 
test = ce_target_encoder.transform(test)

### Hyperparameter tuning:

In [104]:
%timeit
rf = RandomForestRegressor(n_estimators = 100, oob_score = True,n_jobs = 1,random_state =1)
rf.fit(train,train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
                      oob_score=True, random_state=1, verbose=0,
                      warm_start=False)

In [110]:
%%time
# creating a list of different leaf sizes
sample_leaf_options = [1,2,3,4,5,10,20]

# for loop to iterate for each leaf size
for leaf_size in sample_leaf_options :
    model = RandomForestClassifier(n_estimators = 200, n_jobs = -1,random_state =50, min_samples_leaf = leaf_size)
    model.fit(train,train_y)
    print("\n Leaf size :", leaf_size)
    print ("AUC - ROC : ", metrics.roc_auc_score(train_y,model.predict(train)))


 Leaf size : 1
AUC - ROC :  1.0

 Leaf size : 2
AUC - ROC :  0.9945985243709548

 Leaf size : 3
AUC - ROC :  0.958845796530168

 Leaf size : 4
AUC - ROC :  0.9160473653243719

 Leaf size : 5
AUC - ROC :  0.8776393478149508

 Leaf size : 10
AUC - ROC :  0.7702702190890418

 Leaf size : 20
AUC - ROC :  0.6980140070048023
Wall time: 8min 26s
